In [1]:
import numpy as np
import pandas as pd
from kaggle.competitions import nflrush
from tqdm import tqdm_notebook
import torch
from torch import nn, optim
from torch.optim.optimizer import Optimizer, required
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, SELU, LeakyReLU
import torch.nn.functional as F
from tqdm import tqdm_notebook,tqdm,trange
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy.spatial import Voronoi, voronoi_plot_2d
import statsmodels.api as sm
from scipy.interpolate import interp1d
import math
import random
import os
from scipy.stats import trim_mean
import scipy
from copy import deepcopy
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings
import time
start_time = time.time()
time_limit = 9000
warnings.filterwarnings("ignore")
env = nflrush.make_env()

In [2]:
random.seed(721)
os.environ['PYTHONHASHSEED'] = str(831)
np.random.seed(101)
torch.manual_seed(1111)
torch.cuda.manual_seed(1117)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == "cuda:0":
    torch.cuda.set_device(0)
    debug = True
else:
    debug = False
debug = False
device

device(type='cpu')

In [4]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        #inlining this saves 1 second per epoch (V100 GPU) vs having a temp x and then returning x(!)
        return x *( torch.tanh(F.softplus(x)))
    
def gelu(x):
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return gelu
    else:
        raise RuntimeError("activation should be relu/gelu, not %s." % activation)

def multi_head_attention_forward(query,                           # type: Tensor
                                 key,                             # type: Tensor
                                 value,                           # type: Tensor
                                 embed_dim_to_check,              # type: int
                                 num_heads,                       # type: int
                                 in_proj_weight,                  # type: Tensor
                                 in_proj_bias,                    # type: Tensor
                                 bias_k,                          # type: Optional[Tensor]
                                 bias_v,                          # type: Optional[Tensor]
                                 add_zero_attn,                   # type: bool
                                 dropout_p,                       # type: float
                                 out_proj_weight,                 # type: Tensor
                                 out_proj_bias,                   # type: Tensor
                                 training=True,                   # type: bool
                                 key_padding_mask=None,           # type: Optional[Tensor]
                                 need_weights=True,               # type: bool
                                 attn_mask=None,                  # type: Optional[Tensor]
                                 use_separate_proj_weight=False,  # type: bool
                                 q_proj_weight=None,              # type: Optional[Tensor]
                                 k_proj_weight=None,              # type: Optional[Tensor]
                                 v_proj_weight=None,              # type: Optional[Tensor]
                                 static_k=None,                   # type: Optional[Tensor]
                                 static_v=None                    # type: Optional[Tensor]
                                 ):
    # type: (...) -> Tuple[Tensor, Optional[Tensor]]
    r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        embed_dim_to_check: total dimension of the model.
        num_heads: parallel attention heads.
        in_proj_weight, in_proj_bias: input projection weight and bias.
        bias_k, bias_v: bias of the key and value sequences to be added at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        dropout_p: probability of an element to be zeroed.
        out_proj_weight, out_proj_bias: the output projection weight and bias.
        training: apply dropout if is ``True``.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. This is an binary mask. When the value is True,
            the corresponding value on the attention layer will be filled with -inf.
        need_weights: output attn_output_weights.
        attn_mask: mask that prevents attention to certain positions. This is an additive mask
            (i.e. the values will be added to the attention layer).
        use_separate_proj_weight: the function accept the proj. weights for query, key,
            and value in differnt forms. If false, in_proj_weight will be used, which is
            a combination of q_proj_weight, k_proj_weight, v_proj_weight.
        q_proj_weight, k_proj_weight, v_proj_weight, in_proj_bias: input projection weight and bias.
        static_k, static_v: static key and value used for attention operators.


    Shape:
        Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)`, ByteTensor, where N is the batch size, S is the source sequence length.
        - attn_mask: :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
        - static_k: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        - static_v: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.

        Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
    """

    qkv_same = torch.equal(query, key) and torch.equal(key, value)
    kv_same = torch.equal(key, value)

    tgt_len, bsz, embed_dim = query.size()
    assert embed_dim == embed_dim_to_check
    assert list(query.size()) == [tgt_len, bsz, embed_dim]
    assert key.size() == value.size()

    head_dim = embed_dim // num_heads
    assert head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
    scaling = float(head_dim) ** -0.5

    if use_separate_proj_weight is not True:
        if qkv_same:
            # self-attention
            q, k, v = F.linear(query, in_proj_weight, in_proj_bias).chunk(3, dim=-1)

        elif kv_same:
            # encoder-decoder attention
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = F.linear(query, _w, _b)

            if key is None:
                assert value is None
                k = None
                v = None
            else:

                # This is inline in_proj function with in_proj_weight and in_proj_bias
                _b = in_proj_bias
                _start = embed_dim
                _end = None
                _w = in_proj_weight[_start:, :]
                if _b is not None:
                    _b = _b[_start:]
                k, v = F.linear(key, _w, _b).chunk(2, dim=-1)

        else:
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = F.linear(query, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim
            _end = embed_dim * 2
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            k = F.linear(key, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim * 2
            _end = None
            _w = in_proj_weight[_start:, :]
            if _b is not None:
                _b = _b[_start:]
            v = F.linear(value, _w, _b)
    else:
        q_proj_weight_non_opt = torch.jit._unwrap_optional(q_proj_weight)
        len1, len2 = q_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == query.size(-1)

        k_proj_weight_non_opt = torch.jit._unwrap_optional(k_proj_weight)
        len1, len2 = k_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == key.size(-1)

        v_proj_weight_non_opt = torch.jit._unwrap_optional(v_proj_weight)
        len1, len2 = v_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == value.size(-1)

        if in_proj_bias is not None:
            q = F.linear(query, q_proj_weight_non_opt, in_proj_bias[0:embed_dim])
            k = F.linear(key, k_proj_weight_non_opt, in_proj_bias[embed_dim:(embed_dim * 2)])
            v = F.linear(value, v_proj_weight_non_opt, in_proj_bias[(embed_dim * 2):])
        else:
            q = F.linear(query, q_proj_weight_non_opt, in_proj_bias)
            k = F.linear(key, k_proj_weight_non_opt, in_proj_bias)
            v = F.linear(value, v_proj_weight_non_opt, in_proj_bias)
    q = q * scaling

    if bias_k is not None and bias_v is not None:
        if static_k is None and static_v is None:
            k = torch.cat([k, bias_k.repeat(1, bsz, 1)])
            v = torch.cat([v, bias_v.repeat(1, bsz, 1)])
            if attn_mask is not None:
                attn_mask = torch.cat([attn_mask,
                                      torch.zeros((attn_mask.size(0), 1),
                                                  dtype=attn_mask.dtype,
                                                  device=attn_mask.device)], dim=1)
            if key_padding_mask is not None:
                key_padding_mask = torch.cat(
                    [key_padding_mask, torch.zeros((key_padding_mask.size(0), 1),
                                                   dtype=key_padding_mask.dtype,
                                                   device=key_padding_mask.device)], dim=1)
        else:
            assert static_k is None, "bias cannot be added to static key."
            assert static_v is None, "bias cannot be added to static value."
    else:
        assert bias_k is None
        assert bias_v is None

    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    if k is not None:
        k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    if v is not None:
        v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)

    if static_k is not None:
        assert static_k.size(0) == bsz * num_heads
        assert static_k.size(2) == head_dim
        k = static_k

    if static_v is not None:
        assert static_v.size(0) == bsz * num_heads
        assert static_v.size(2) == head_dim
        v = static_v

    src_len = k.size(1)

    if key_padding_mask is not None:
        assert key_padding_mask.size(0) == bsz
        assert key_padding_mask.size(1) == src_len

    if add_zero_attn:
        src_len += 1
        k = torch.cat([k, torch.zeros((k.size(0), 1) + k.size()[2:], dtype=k.dtype, device=k.device)], dim=1)
        v = torch.cat([v, torch.zeros((v.size(0), 1) + v.size()[2:], dtype=v.dtype, device=v.device)], dim=1)
        if attn_mask is not None:
            attn_mask = torch.cat([attn_mask, torch.zeros((attn_mask.size(0), 1),
                                                          dtype=attn_mask.dtype,
                                                          device=attn_mask.device)], dim=1)
        if key_padding_mask is not None:
            key_padding_mask = torch.cat(
                [key_padding_mask, torch.zeros((key_padding_mask.size(0), 1),
                                               dtype=key_padding_mask.dtype,
                                               device=key_padding_mask.device)], dim=1)

    attn_output_weights = torch.bmm(q, k.transpose(1, 2))
    assert list(attn_output_weights.size()) == [bsz * num_heads, tgt_len, src_len]

    if attn_mask is not None:
        attn_mask = attn_mask.unsqueeze(0)
        attn_output_weights += attn_mask

    if key_padding_mask is not None:
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        attn_output_weights = attn_output_weights.masked_fill(
            key_padding_mask.unsqueeze(1).unsqueeze(2),
            float('-inf'),
        )
        attn_output_weights = attn_output_weights.view(bsz * num_heads, tgt_len, src_len)

    attn_output_weights = F.softmax(
        attn_output_weights, dim=-1)
    attn_output_weights = F.dropout(attn_output_weights, p=dropout_p, training=training)

    attn_output = torch.bmm(attn_output_weights, v)
    assert list(attn_output.size()) == [bsz * num_heads, tgt_len, head_dim]
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = F.linear(attn_output, out_proj_weight, out_proj_bias)

    if need_weights:
        # average attention weights over heads
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        return attn_output, attn_output_weights.sum(dim=1) / num_heads
    else:
        return attn_output, None

        
class MultiheadAttention(nn.Module):
    r"""Allows the model to jointly attend to information
    from different representation subspaces.
    See reference: Attention Is All You Need

    .. math::
        \text{MultiHead}(Q, K, V) = \text{Concat}(head_1,\dots,head_h)W^O
        \text{where} head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)

    Args:
        embed_dim: total dimension of the model.
        num_heads: parallel attention heads.
        dropout: a Dropout layer on attn_output_weights. Default: 0.0.
        bias: add bias as module parameter. Default: True.
        add_bias_kv: add bias to the key and value sequences at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        kdim: total number of features in key. Default: None.
        vdim: total number of features in key. Default: None.

        Note: if kdim and vdim are None, they will be set to embed_dim such that
        query, key, and value have the same number of features.

    Examples::

        >>> multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        >>> attn_output, attn_output_weights = multihead_attn(query, key, value)
    """

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None):
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        self.in_proj_weight = Parameter(torch.empty(3 * embed_dim, embed_dim))

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.Tensor(embed_dim, embed_dim))
            self.k_proj_weight = Parameter(torch.Tensor(embed_dim, self.kdim))
            self.v_proj_weight = Parameter(torch.Tensor(embed_dim, self.vdim))

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim))
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)

        if add_bias_kv:
            self.bias_k = Parameter(torch.empty(1, 1, embed_dim))
            self.bias_v = Parameter(torch.empty(1, 1, embed_dim))
        else:
            self.bias_k = self.bias_v = None

        self.add_zero_attn = add_zero_attn

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            xavier_normal_(self.bias_k)
        if self.bias_v is not None:
            xavier_normal_(self.bias_v)

    def forward(self, query, key, value, key_padding_mask=None,
                need_weights=True, attn_mask=None):
        r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. This is an binary mask. When the value is True,
            the corresponding value on the attention layer will be filled with -inf.
        need_weights: output attn_output_weights.
        attn_mask: mask that prevents attention to certain positions. This is an additive mask
            (i.e. the values will be added to the attention layer).  

    Shape:
        - Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)`, ByteTensor, where N is the batch size, S is the source sequence length.
        - attn_mask: :math:`(L, S)` where L is the target sequence length, S is the source sequence length.

        - Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
        """
        if hasattr(self, '_qkv_same_embed_dim') and self._qkv_same_embed_dim is False:
            return multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias, 
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights, 
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight)
        else:
            if not hasattr(self, '_qkv_same_embed_dim'):
                warnings.warn('A new version of MultiheadAttention module has been implemented. \
                    Please re-train your model with the new module',
                              UserWarning)

            return multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias, 
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights, 
                attn_mask=attn_mask)
        
class TransformerEncoderLayer(nn.Module):
    r"""TransformerEncoderLayer is made up of self-attn and feedforward network.
    This standard encoder layer is based on the paper "Attention Is All You Need".
    Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
    Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in
    Neural Information Processing Systems, pages 6000-6010. Users may modify or implement
    in a different way during application.

    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of intermediate layer, relu or gelu (default=relu).

    Examples::
        >>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        >>> src = torch.rand(10, 32, 512)
        >>> out = encoder_layer(src)
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
#         self.activation = Mish()

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        r"""Pass the input through the endocder layer.

        Args:
            src: the sequnce to the encoder layer (required).
            src_mask: the mask for the src sequence (optional).
            src_key_padding_mask: the mask for the src keys per batch (optional).

        Shape:
            see the docs in Transformer class.
        """
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        if hasattr(self, "activation"):
            src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        else:  # for backward compatibility
            src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src
    
class TransformerDecoderLayer(nn.Module):
    r"""TransformerDecoderLayer is made up of self-attn, multi-head-attn and feedforward network.
    This standard decoder layer is based on the paper "Attention Is All You Need".
    Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
    Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in
    Neural Information Processing Systems, pages 6000-6010. Users may modify or implement
    in a different way during application.

    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of intermediate layer, relu or gelu (default=relu).

    Examples::
        >>> decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
        >>> memory = torch.rand(10, 32, 512)
        >>> tgt = torch.rand(20, 32, 512)
        >>> out = decoder_layer(tgt, memory)
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        r"""Pass the inputs (and mask) through the decoder layer.

        Args:
            tgt: the sequence to the decoder layer (required).
            memory: the sequnce from the last layer of the encoder (required).
            tgt_mask: the mask for the tgt sequence (optional).
            memory_mask: the mask for the memory sequence (optional).
            tgt_key_padding_mask: the mask for the tgt keys per batch (optional).
            memory_key_padding_mask: the mask for the memory keys per batch (optional).

        Shape:
            see the docs in Transformer class.
        """
        tgt2 = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2 = self.multihead_attn(tgt, memory, memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        if hasattr(self, "activation"):
            tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        else:  # for backward compatibility
            tgt2 = self.linear2(self.dropout(F.relu(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt
    
    
class RAdam(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss
class Lookahead(Optimizer):
    def __init__(self, optimizer, k=5, alpha=0.5):
        self.optimizer = optimizer
        self.k = k
        self.alpha = alpha
        self.param_groups = self.optimizer.param_groups
        self.state = defaultdict(dict)
        self.fast_state = self.optimizer.state
        for group in self.param_groups:
            group["counter"] = 0
    
    def update(self, group):
        for fast in group["params"]:
            param_state = self.state[fast]
            if "slow_param" not in param_state:
                param_state["slow_param"] = torch.zeros_like(fast.data)
                param_state["slow_param"].copy_(fast.data)
            slow = param_state["slow_param"]
            slow += (fast.data - slow) * self.alpha
            fast.data.copy_(slow)
    
    def update_lookahead(self):
        for group in self.param_groups:
            self.update(group)

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        for group in self.param_groups:
            if group["counter"] == 0:
                self.update(group)
            group["counter"] += 1
            if group["counter"] >= self.k:
                group["counter"] = 0
        return loss

    def state_dict(self):
        fast_state_dict = self.optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict["state"]
        param_groups = fast_state_dict["param_groups"]
        return {
            "fast_state": fast_state,
            "slow_state": slow_state,
            "param_groups": param_groups,
        }

    def load_state_dict(self, state_dict):
        slow_state_dict = {
            "state": state_dict["slow_state"],
            "param_groups": state_dict["param_groups"],
        }
        fast_state_dict = {
            "state": state_dict["fast_state"],
            "param_groups": state_dict["param_groups"],
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.optimizer.load_state_dict(fast_state_dict)
        self.fast_state = self.optimizer.state

    def add_param_group(self, param_group):
        param_group["counter"] = 0
        self.optimizer.add_param_group(param_group)

In [5]:
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

In [6]:
num_cols = ['IsRusher',
            'IsRusherTeam',
            'X','Y',
            'Dir_X','Dir_Y',
            'Orientation_X','Orientation_Y',
            'S',
            'DistanceToBall',
            'BallDistanceX','BallDistanceY',
            'BallAngleX','BallAngleY',
            'related_horizontal_v','related_vertical_v',
            'related_horizontal_A','related_vertical_A',
#             'horizontal_v','vertical_v',
#             'horizontal_A','vertical_A',
            'TeamDistance','EnermyTeamDistance',
            'TeamXstd','EnermyXstd',
            'EnermyYstd','TeamYstd',
            'DistanceToBallRank',
            'DistanceToBallRank_AttTeam','DistanceToBallRank_DefTeam',
            'YardLine',
            'NextX','NextY',
            'NextDistanceToBall',
            'BallNextAngleX','BallNextAngleY',
            'BallNextDistanceX','BallNextDistanceY',
            'A',
#             'Dis0',
#             'Season',
#             'problem_pos',
#             'PlayDirection',
#             '1YardToYardLine',
#             'AttCloseYardLine','DefCloseYardLine'
           ]

map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
for abb in train_df['PossessionTeam'].unique():
    map_abbr[abb] = abb

# problem_pos = ['SS','FS','CB','WR']
    
def Create_group_features(df,train=True):
    df['PossessionTeam'] = df['PossessionTeam'].map(map_abbr)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].map(map_abbr)
    df['Season'] = (df['Season'] == 2017)
    df.loc[df['Season'],'A'] = np.nan
    df['S'] = 10*df['Dis']
#     df.loc[df['Season']&(df['Dis']==0),'S']=np.nan
#     df['Dis0'] = df['Dis'] == 0
#     df.loc[df['Dis0'],'S']=np.nan
#     df.loc[df['Dis0'],'A']=np.nan
    df.loc[df['Season'],'Orientation'] = np.mod(90+df.loc[df['Season'],'Orientation'],360)
    mask = df['PlayDirection'] == 'left'
    df.loc[mask, 'X'] = 120 - df.loc[mask, 'X']
    df.loc[mask, 'Y'] = (160 / 3) - df.loc[mask, 'Y']
    df.loc[mask, 'Orientation'] = np.mod(180 + df.loc[mask, 'Orientation'], 360)
    df.loc[mask, 'Dir'] = np.mod(180 + df.loc[mask, 'Dir'], 360)
    df['FieldPosition'].fillna('', inplace=True)
    mask = df['PossessionTeam'] != df['FieldPosition']
    df.loc[mask, 'YardLine'] = 100 - df.loc[mask, 'YardLine']
#     df['1YardToYardLine'] = (df['X'] - (10+df['YardLine'])).abs() < 1
    df['PlayDirection'] = df['PlayDirection'].map({'left':1,'right':0})
    df['Team'] = df['Team'].map({'away':1,'home':0})
    df['Dir'] = np.mod(90+df['Dir'],360)
    df['Orientation'] = np.mod(90+df['Orientation'],360)
    df['Orientation_X'] = np.cos(df['Orientation']*2*np.pi/360)
    df['Orientation_Y'] = np.sin(df['Orientation']*2*np.pi/360)
    df['Dir_X'] = np.cos(df['Dir']*2*np.pi/360)
    df['Dir_Y'] = np.sin(df['Dir']*2*np.pi/360)
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    rushers = df[df['IsRusher']][['PlayId', 'Team', 'X', 'Y', 'A', 'S', 'Dir_X','Dir_Y']]
    rushers.columns = ['PlayId', 'RusherTeam', 'BallX', 'BallY', 'BallA', 'BallS', 'BallDir_X', 'BallDir_Y']
    df = df.join(rushers.set_index('PlayId'), on='PlayId')
    df['IsRusherTeam'] = (df['Team'] == df['RusherTeam'])
    df['DistanceToBall'] = ((df['X'] - df['BallX'])**2 + (df['Y'] - df['BallY'])**2 )**0.5
    dx = df['BallX']-df['X']
    dy = df['BallY']-df['Y']
    angle = np.arctan(dy/(1e-8+dx))*360/(2*np.pi)
    angle[dx<0] += 180
    angle[(dx>0)&(dy<0)] += 360
    angle = np.mod(angle,360)
    df['BallAngleX'] = np.sin(angle*2*np.pi/360)
    df['BallAngleY'] = np.cos(angle*2*np.pi/360)
    df['BallDistanceX'] = -dx
    df['BallDistanceY'] = -dy
    df['related_horizontal_v'] = df['S']*df['Dir_X'] - df['BallS']*df['BallDir_X']
    df['related_vertical_v'] = df['S']*df['Dir_Y'] - df['BallS']*df['BallDir_Y']
    df['related_horizontal_A'] = df['A']*df['Dir_X'] - df['BallA']*df['BallDir_X']
    df['related_vertical_A'] = df['A']*df['Dir_Y'] - df['BallA']*df['BallDir_Y']
    df['DistanceToBallRank'] = df.groupby('PlayId')['DistanceToBall'].rank()
    Distance_Ball_Team = df.groupby(['PlayId','Team'])['DistanceToBall'].rank()
    df['DistanceToBallRank_AttTeam'] = np.nan
    df['DistanceToBallRank_DefTeam'] = np.nan
    df.loc[df['IsRusherTeam'],'DistanceToBallRank_AttTeam'] = Distance_Ball_Team.loc[df['IsRusherTeam']]
    df.loc[~df['IsRusherTeam'],'DistanceToBallRank_DefTeam'] = Distance_Ball_Team.loc[~df['IsRusherTeam']]
    TeamXY = df.groupby(['PlayId','Team'])[['X','Y']].transform(np.mean)
    df['TeamDistance'] = ((df['X'] - TeamXY['X'])**2 + (df['Y'] - TeamXY['Y'])**2 )**0.5
    df['TeamXstd'] = df['X'] - TeamXY['X']
    df['TeamYstd'] = df['Y'] - TeamXY['Y']
    EnermyTeamXY = TeamXY.groupby(df['PlayId']).apply(lambda x: x.iloc[::-1]).values
    EnermyTeamXY = pd.DataFrame(EnermyTeamXY,index = TeamXY.index,columns = TeamXY.columns)
    df['EnermyTeamDistance'] = ((df['X'] - EnermyTeamXY['X'])**2 + (df['Y'] - EnermyTeamXY['Y'])**2 )**0.5
    df['EnermyXstd'] = df['X'] - EnermyTeamXY['X']
    df['EnermyYstd'] = df['Y'] - EnermyTeamXY['Y']
    df['NextX'] = df['S']*df['Dir_X']+df['X']
    df['NextY'] = df['S']*df['Dir_Y']+df['Y']
    rushers = df[df['IsRusher']][['PlayId', 'NextX', 'NextY']]
    rushers.columns = ['PlayId', 'BallNextX', 'BallNextY']
    df = df.join(rushers.set_index('PlayId'), on='PlayId')
    df['NextDistanceToBall'] = ((df['NextX'] - df['BallNextX'])**2 + (df['NextY'] - df['BallNextY'])**2 )**0.5
    dx = df['BallNextX']-df['NextX']
    dy = df['BallNextY']-df['NextY']
    angle = np.arctan(dy/(1e-8+dx))*360/(2*np.pi)
    angle[dx<0] += 180
    angle[(dx>0)&(dy<0)] += 360
    angle = np.mod(angle,360)
    df['BallNextAngleX'] = np.sin(angle*2*np.pi/360)
    df['BallNextAngleY'] = np.cos(angle*2*np.pi/360)
    df['BallNextDistanceX'] = -dx
    df['BallNextDistanceY'] = -dy
#     df['problem_pos'] = df['Position'].isin(problem_pos)
#     YL_Dis = df.groupby(['PlayId','IsRusherTeam'])['1YardToYardLine'].transform(np.sum)
#     df['AttCloseYardLine'] = np.nan
#     df['DefCloseYardLine'] = np.nan
#     df.loc[df['IsRusherTeam'],'AttCloseYardLine'] = YL_Dis.loc[df['IsRusherTeam']]
#     df.loc[~df['IsRusherTeam'],'DefCloseYardLine'] = YL_Dis.loc[~df['IsRusherTeam']]
    
#     df['horizontal_v']=df['S']*df['Dir_X']
#     df['vertical_v']=df['S']*df['Dir_Y']
#     df['horizontal_A']=df['A']*df['Dir_X']
#     df['vertical_A']=df['A']*df['Dir_Y']
#     df['SA']=df['S']*df['A']
    return df[num_cols]

In [7]:
%%time
groups = (train_df['GameId'].astype(str).str[:4] + train_df['Week'].astype(str))
groups = groups.groupby(train_df['PlayId']).apply(lambda x:x.iloc[0]).values
train_y = train_df['Yards'].groupby(train_df['PlayId']).apply(lambda x:x.iloc[0]).values
train_X_df = Create_group_features(train_df.copy())
bool_cols = ['IsRusher','IsRusherTeam']
Scale_cols = [col for col in train_X_df.columns if col not in bool_cols]
X_mean = train_X_df[Scale_cols].mean()
X_std = train_X_df[Scale_cols].std()
train_X = train_X_df.copy()
train_X[bool_cols] = train_X[bool_cols].astype(np.float32)
train_X[Scale_cols] = (train_X_df[Scale_cols] - X_mean[Scale_cols])/X_std[Scale_cols]
train_X.fillna(0,inplace=True)
train_X = train_X.values.reshape(len(train_y),22,-1)
train_X.shape

CPU times: user 26.5 s, sys: 4.04 s, total: 30.5 s
Wall time: 28 s


(23171, 22, 36)

In [8]:
train_X_df

,IsRusher,IsRusherTeam,X,Y,Dir_X,Dir_Y,Orientation_X,Orientation_Y,S,DistanceToBall,...,DistanceToBallRank_DefTeam,YardLine,NextX,NextY,NextDistanceToBall,BallNextAngleX,BallNextAngleY,BallNextDistanceX,BallNextDistanceY,A
0,False,False,46.09,18.493333,0.049198,0.998789,0.139346,0.990244,4.0,6.480872,...,4.0,35,46.286794,22.488489,8.705788,0.215510,-0.976502,8.501217,-1.876180,NaN
1,False,False,45.33,20.693333,-0.320613,0.947210,0.886123,0.463451,0.1,4.593310,...,1.0,35,45.297939,20.788054,8.320322,0.429865,-0.902893,7.512362,-3.576615,NaN
2,False,False,46.00,20.133333,-0.386389,0.922336,0.998620,0.052510,3.1,5.448982,...,3.0,35,44.802194,22.992575,7.149515,0.191914,-0.981412,7.016617,-1.372095,NaN
3,False,False,48.54,25.633333,0.962975,0.269592,0.999992,-0.004014,0.2,7.820038,...,6.0,35,48.732595,25.687252,11.026624,-0.119944,-0.992781,10.947018,1.322582,NaN
4,False,False,50.68,17.913333,0.270432,0.962739,0.975802,0.218654,1.6,10.622476,...,8.0,35,51.112692,19.453716,14.203150,0.345765,-0.938321,13.327115,-4.910954,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509757,False,True,86.77,24.200000,-0.264041,0.964511,-0.701531,0.712639,2.2,6.345345,...,NaN,77,86.189109,26.321925,9.586588,-0.219178,-0.975685,9.353489,2.101172,2.12
509758,False,True,86.76,27.180000,-0.679441,0.733730,-0.805204,0.592997,1.1,6.017516,...,NaN,77,86.012615,27.987103,9.919809,-0.379680,-0.925118,9.176995,3.766349,0.66
509759,False,True,87.26,27.050000,-0.360322,0.932828,-0.069060,0.997613,2.6,6.497815,...,NaN,77,86.323162,29.475352,10.845472,-0.484497,-0.874793,9.487542,5.254599,1.18
509760,False,True,84.57,24.370000,0.058261,-0.998301,-0.528735,-0.848787,4.7,4.258321,...,NaN,77,84.843827,19.677983,9.206961,0.493406,-0.869799,8.008207,-4.542770,1.79


In [9]:
n = np.arange(-99, 100)
n = np.row_stack([n] * train_y.shape[0])
ym = train_y.reshape(train_y.shape[0], 1)
train_y = np.heaviside(n - ym, 1)

In [10]:
Lower_yards = np.heaviside(99 - train_X_df['YardLine'].values[::22].reshape(train_y.shape[0], 1) - np.row_stack([np.arange(199)] * train_y.shape[0]),0)
Upper_yards = np.heaviside(np.row_stack([np.arange(199)] * train_y.shape[0]) - 199 + train_X_df['YardLine'].values[::22].reshape(train_y.shape[0], 1),0)
Yard_mask = Lower_yards + Upper_yards

In [11]:
train_y_OH = np.zeros([train_y.shape[0],199])
train_y_OH[np.arange(len(ym)), (ym+99).reshape(-1)] = 1

In [12]:
class Model(nn.Module):
    def __init__(self,params):
        super(Model, self).__init__()
        Activation = params['Activation']
        self.layers = params['layer']
        
        self.emb = Seq(nn.Dropout(params['dropout_0']),Lin(params['input_size'],params['hidden_0']))
#         self.emb = Seq(nn.Dropout(params['dropout_0']),Lin(params['input_size'],params['hidden_3']),Activation,Lin(params['hidden_3'],params['hidden_0']))
        self.transformer_encoder = nn.ModuleList()
        self.transformer_decoder = nn.ModuleList()
        for l in range(self.layers):
            self.transformer_encoder.append(TransformerEncoderLayer(params['hidden_0'],\
                                                       nhead=params['head'],\
                                                       dim_feedforward=params['hidden_1'],\
                                                       dropout=params['dropout_1'],\
                                                       activation = params['transformer_activation']))
            self.transformer_decoder.append(TransformerDecoderLayer(params['hidden_0'],\
                                                   nhead=params['head'],\
                                                   dim_feedforward=params['hidden_1'],\
                                                   dropout=params['dropout_1'],\
                                                   activation = params['transformer_activation']))

        self.mlp = Seq(Lin(params['hidden_0'], params['hidden_3']),Activation,nn.Dropout(params['dropout_3']),Lin(params['hidden_3'], 199))

    def forward(self, X):
        rusher_mask = X[:,:,0]>0
        combine = self.emb(X)
        x = combine.permute(1,0,2)
        for l in range(self.layers):
            x = self.transformer_encoder[l](x)
        rusher = (x.permute(1,0,2)[rusher_mask].unsqueeze(1)).permute(1,0,2)
        for l in range(self.layers):
            rusher = self.transformer_decoder[l](rusher,x)
        x = rusher.permute(1,0,2)
        y = x.squeeze(1)
        y = self.mlp(y)
        y = F.softmax(y,dim=-1)
        return y,torch.cumsum(y,dim = -1)

In [13]:
def Schedule_scheme(x):
    if x%3==0:
        return 1
    elif x%3==1:
        return 0.5
    else:
        return 0.2

def Create_model(params,input_size = train_X.shape[-1]):
    params['input_size'] = input_size
    clf = Model(params).to(device)
    vv = 0
    for v in list(clf.parameters()):
        vv += len(v.view(-1))
    print("Num_Variables:",vv)
    base_opt = RAdam(clf.parameters(),lr=params['lr'],\
                     weight_decay=params['weight_decay'],\
                     betas=(1-params['alpha'], 1-params['beta']), eps=params['eps']
                    )
    optimizer = Lookahead(base_opt, k=params['lookahead_k'], alpha=params['lookahead_alpha'])
#     scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda= lambda x:  0.5*(1 + np.cos(np.pi*np.mod(x,params['cosine_T'])/params['cosine_T'])))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda= lambda x: 1 if x%2==0 else 0.5)
#     scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda= Schedule_scheme)
    return clf,optimizer,scheduler

In [14]:
config = {'Activation': 'lrelu1',
          'alpha': 0.06177912913861566,
          'batch_size': 82,
          'beta': 0.025662159487370977,
          'cosine_T': 67,
          'dropout_0': 0.023898897998374913,
          'dropout_1': 0.06080646510619758,
          'dropout_2': 0.025011299009768506,
          'dropout_3': 0.22165717850445166,
          'eps': 5.884062604808768e-07,
          'head': 6,
          'hidden_0': 111,
          'hidden_1': 2040,
          'hidden_2': 1742,
          'hidden_3': 933,
          'layer': 4,
          'lookahead_alpha': 0.5422623511158232,
          'lookahead_k': 3,
          'loss_1': 0.48305609964511376,
          'loss_2': 0.2353079388007755,
          'lr': 0.00014789689708999082,
          'min_lr': 0.0038268377963604707,
          'transformer_activation': 'gelu',
          'weight_decay': 2.2445535063229495e-05}

act_map = {'lrelu1':LeakyReLU(0.1),
           'lrelu2':LeakyReLU(0.2),
           'relu':ReLU(),
           'selu':SELU(),
           'prelu':nn.PReLU(),
           'elu':nn.ELU(),
           'mish':Mish()}

params = {'Activation': Mish(),
              'dropout_0': config['dropout_0'],
              'hidden_0': 128,
              'head': 32,
              'hidden_1': 512,
              "dropout_1": 0.2,
              'transformer_activation': config['transformer_activation'],
              "hidden_2": config['hidden_2'],
              "dropout_2": config['dropout_2'],
              'hidden_3': 512,
              'dropout_3': 0.3,
              "layer": 2,
              "lr": 5*config['lr'],
              'weight_decay': 1e-1,
              'alpha': config['alpha'],
              'beta': config['beta'],
              'eps': config['eps'],
              'lookahead_k': 5,
              'lookahead_alpha': 0.5,
              'cosine_T': config['cosine_T'],
              'min_lr': 0.01*config['lr'] * config['min_lr'],
              'batch_size': 32,
              'loss_1': 0.5,
              'loss_2': 0.5
             }

In [15]:
# #CV
# from sklearn.model_selection import GroupKFold,KFold
# groups = (train_df['GameId'].astype(str).str[:4] + train_df['Week'].astype(str))
# common_idx = np.where((train_df['Season'] == 2017).values[::22])[0]
# groups = groups[train_df['Season'] != 2017].values[::22]
# kf = GroupKFold(n_splits=3)
# model_list = []
# iso_reg = []
# oof = np.zeros([len(train_X),199])
# oof_iso = np.zeros([len(train_X),199])
# batch_size = params['batch_size']
# for fold_, (trn_idx_, val_idx_) in enumerate(kf.split(train_y[len(common_idx):], train_y[len(common_idx):],groups)):
#     print("fold n°{}".format(fold_))
#     trn_idx = np.append(common_idx,trn_idx_+len(common_idx))
#     val_idx = val_idx_+len(common_idx)
#     X_tr,y_tr,y_OH_tr = train_X[trn_idx], train_y[trn_idx],train_y_OH[trn_idx]
#     X_val,y_val,y_OH_val = train_X[val_idx], train_y[val_idx],train_y_OH[val_idx]
    
#     tmp = deepcopy(y_OH_tr)
#     y_OH_tr[:,:-1] += 0.5 * tmp[:,1:]
#     y_OH_tr[:,1:] += 0.5 * tmp[:,:-1]
    
#     params['cosine_T'] = math.ceil(len(y_tr)/batch_size)
#     clf,optimizer,scheduler = Create_model(params)
#     best_loss = 9999
#     Best_test_pred = []
#     for epoch in tqdm_notebook(range(30)):
#         clf.train()
#         batches = np.arange(len(trn_idx))
#         np.random.shuffle(batches)
#         batches = batches[:batch_size * (len(batches)//batch_size)]
#         batches = batches.reshape(-1,batch_size)
#         Train_pred = []
#         MA_loss = 0
#         for batch in batches:
#             optimizer.zero_grad()
#             X = torch.Tensor(X_tr[batch]).to(device)
#             y = torch.Tensor(y_tr[batch]).to(device)
#             y_OH = torch.Tensor(y_OH_tr[batch]).to(device)
#             y_pred_OH,y_pred = clf(X)
#             loss = torch.mean((y - y_pred)**2)
# #             loss2 = -torch.mean(y_OH * torch.log(1e-8 + y_pred_OH) + (1-y_OH) * torch.log(1-y_pred_OH+1e-8))
#             loss2 = -torch.mean(y_OH * torch.log(1e-8 + y_pred_OH) + (1-y_OH) * torch.log(1-y_pred_OH+1e-8))
# #             loss3 = torch.mean((y_OH - y_pred_OH)**2)
# #             l2loss = ((clf.mlp[-1].weight[1:] - clf.mlp[-1].weight[:-1])**2).mean()
# #             (params['loss_1']*loss+params['loss_2']*loss2 + l2loss).backward()
#             (params['loss_1']*loss+params['loss_2']*loss2).backward()
# #             mu,y_pred = clf(X)
# #             loss = torch.mean((y - y_pred)**2)
# #             loss2 = (torch.abs(mu - (y.argmax(-1).float()-99.5)/57.445626465380286)).mean()
# #             (0.5*loss+0.5*loss2).backward()
#             optimizer.step()
#             Train_pred.append(y_pred.detach().cpu().reshape(-1))
#             MA_loss += loss.item()
#         if epoch >= 10:
#             scheduler.step()
#         Train_pred = torch.cat(Train_pred,dim=0).numpy()
#         MA_loss /= len(batches)
# #         plt.plot(y_pred_OH.detach().cpu().numpy()[0])
# #         plt.show()
#         with torch.no_grad():
#             clf.eval()
#             batches = np.arange(len(val_idx))
#             last_batch = batches[batch_size * (len(batches)//batch_size):]
#             batches = batches[:batch_size * (len(batches)//batch_size)]
#             batches = batches.reshape(-1,batch_size)
#             Test_loss = 0
#             Test_pred = []
#             for batch in batches:
#                 X = torch.Tensor(X_val[batch]).to(device)
#                 y = torch.Tensor(y_val[batch]).to(device)
#                 y_pred = clf(X)[1]
#                 loss = torch.mean((y - y_pred)**2)
#                 Test_loss += loss.item() * batch_size
#                 Test_pred.append(y_pred.detach().cpu().reshape(-1,199))
#             if len(last_batch) > 0:
#                 batch = last_batch
#                 X = torch.Tensor(X_val[batch]).to(device)
#                 y = torch.Tensor(y_val[batch]).to(device)
#                 y_pred = clf(X)[1]
#                 loss = torch.mean((y - y_pred)**2)
#                 Test_loss += loss.item() * batch_size
#                 Test_pred.append(y_pred.detach().cpu().reshape(-1,199))
#             Test_loss /= len(val_idx)
#             Test_pred = torch.cat(Test_pred,dim=0).numpy()
#             if Test_loss < best_loss:
#                 Best_weight = clf.state_dict()
#                 Best_weight = {name : Best_weight[name].cpu() for name in Best_weight}
#                 best_loss = Test_loss
#             print("epoch",epoch,"Train:",MA_loss,"Test:",Test_loss,"Best:",best_loss)
#         if epoch >= 10 and epoch%2 == 1:
#             Best_test_pred.append(np.array(Test_pred.copy(), dtype=np.float64))
#             model_list.append(deepcopy(clf))
        
#     Best_test_pred = np.mean(Best_test_pred,0)
#     oof[val_idx] = Best_test_pred
#     print(((1 - Yard_mask[val_idx])*(oof[val_idx] - train_y[val_idx])**2).mean())

# print(((1 - Yard_mask[11900:])*(oof[11900:] - train_y[11900:])**2).mean())
# print(((1 - Yard_mask[-3438:])*(oof[-3438:] - train_y[-3438:])**2).mean())

In [16]:
model_list = []
oof = np.zeros([len(train_X),199])
batch_size = params['batch_size']
for fold_ in range(999):
    trn_idx = np.arange(len(train_X))
    X_tr,y_tr,y_OH_tr = train_X[trn_idx], train_y[trn_idx],train_y_OH[trn_idx]
    tmp = deepcopy(y_OH_tr)
    y_OH_tr[:,:-1] += 0.5 * tmp[:,1:]
    y_OH_tr[:,1:] += 0.5 * tmp[:,:-1]
    params['cosine_T'] = len(y_tr)//batch_size
    clf,optimizer,scheduler = Create_model(params)
    best_loss = 9999
    for epoch in tqdm_notebook(range(30)):
        clf.train()
        batches = np.arange(len(trn_idx))
        np.random.shuffle(batches)
        batches = batches[:batch_size * (len(batches)//batch_size)]
        batches = batches.reshape(-1,batch_size)
        Train_pred = []
        MA_loss = 0
        for batch in batches:
            optimizer.zero_grad()
            X = torch.Tensor(X_tr[batch]).to(device)
            y = torch.Tensor(y_tr[batch]).to(device)
            y_OH = torch.Tensor(y_OH_tr[batch]).to(device)
            y_pred_OH,y_pred = clf(X)
            loss = torch.mean((y - y_pred)**2)
            loss2 = -torch.mean(y_OH * torch.log(1e-8 + y_pred_OH) + (1-y_OH) * torch.log(1-y_pred_OH+1e-8))
#             l2loss = ((clf.mlp[-1].weight[1:] - clf.mlp[-1].weight[:-1])**2).mean()
#             (params['loss_1']*loss+params['loss_2']*loss2 + 0.1*l2loss).backward()
            (params['loss_1']*loss+params['loss_2']*loss2).backward()
            optimizer.step()
            MA_loss += loss.item()
        if epoch >= 10:
            scheduler.step()
        MA_loss /= len(batches)
        print("epoch",epoch,"Train:",MA_loss)
        if epoch >= 10 and epoch%2 == 1:
            model_list.append(deepcopy(clf))
        if time.time()-start_time > time_limit:
            break
    if time.time()-start_time > time_limit:
        print("Overtime")
        break

Num_Variables: 1098567


epoch 0 Train: 0.016250557515729035
epoch 1 Train: 0.013002622230055624
epoch 2 Train: 0.012853930019953632
epoch 3 Train: 0.012681736015246471
epoch 4 Train: 0.012624602738474574
epoch 5 Train: 0.012520734660034146
epoch 6 Train: 0.012436979121621184
epoch 7 Train: 0.012421118066574123
epoch 8 Train: 0.012356209761632904
epoch 9 Train: 0.01229370125885847
epoch 10 Train: 0.012211518966752356
epoch 11 Train: 0.012015246002142477
epoch 12 Train: 0.01213232027199531
epoch 13 Train: 0.011954837246509835
epoch 14 Train: 0.012096115759090207
epoch 15 Train: 0.01188300004054005
epoch 16 Train: 0.01203598197194547
epoch 17 Train: 0.011842479867582821
epoch 18 Train: 0.012022248765155261
epoch 19 Train: 0.011746972078941145
epoch 20 Train: 0.011958495079596315
epoch 21 Train: 0.011709122422655519
epoch 22 Train: 0.011908630268051397
epoch 23 Train: 0.011660106068487325
epoch 24 Train: 0.011862349192205385
epoch 25 Train: 0.011580199563428142
epoch 26 Train: 0.011832864719664276
epoch 27 Train:

epoch 0 Train: 0.01618642760929523
epoch 1 Train: 0.013034478323894311
epoch 2 Train: 0.012811590422804144
epoch 3 Train: 0.012682979766958655
epoch 4 Train: 0.012602103955127834
epoch 5 Train: 0.01252070854055227
epoch 6 Train: 0.012471022923703393
epoch 7 Train: 0.012444139569508971
epoch 8 Train: 0.012344851316781251
epoch 9 Train: 0.012293871845252482
epoch 10 Train: 0.012273997669777117
epoch 11 Train: 0.012008431556027445
epoch 12 Train: 0.012202163953051095
epoch 13 Train: 0.011963650407346779
epoch 14 Train: 0.012106919840611106
epoch 15 Train: 0.011884814505825509
epoch 16 Train: 0.012062151657767306
epoch 17 Train: 0.011838405256710165
epoch 18 Train: 0.011999460241438317
epoch 19 Train: 0.011810143858976456
epoch 20 Train: 0.011969439388839374
epoch 21 Train: 0.011726456332838585
epoch 22 Train: 0.011908815189609458
epoch 23 Train: 0.011644204309884233
epoch 24 Train: 0.01189302258741481
epoch 25 Train: 0.01167519184701577
epoch 26 Train: 0.011858866449049288
epoch 27 Train:

epoch 0 Train: 0.016135400747725976
epoch 1 Train: 0.013027783395794544
epoch 2 Train: 0.012842109908596792
epoch 3 Train: 0.012699492830134804
epoch 4 Train: 0.01261301872575172
epoch 5 Train: 0.012516204125597785
epoch 6 Train: 0.012437365163832861
epoch 7 Train: 0.01240759673566964
epoch 8 Train: 0.012308934055145274
epoch 9 Train: 0.012290495526925429
epoch 10 Train: 0.012240547558179547
epoch 11 Train: 0.012043592278223011
epoch 12 Train: 0.012173047585114425
epoch 13 Train: 0.01196403183925444
epoch 14 Train: 0.012118097928543035
epoch 15 Train: 0.011927695988641424
epoch 16 Train: 0.012056570499492244
epoch 17 Train: 0.01178159981062996
epoch 18 Train: 0.012048429518150813
epoch 19 Train: 0.011773253222464362
epoch 20 Train: 0.011981093394827324
epoch 21 Train: 0.011700848439703206
epoch 22 Train: 0.0119448184660888
epoch 23 Train: 0.011680012481359157
Overtime


In [17]:
for (test_df, sample_prediction_df) in tqdm_notebook(env.iter_test()):
    test_X = Create_group_features(test_df.copy(),train=False)
    yardline = test_X['YardLine'].iloc[-1]
    test_X[bool_cols] = test_X[bool_cols].astype(np.float32)
    test_X[Scale_cols] = (test_X[Scale_cols] - X_mean[Scale_cols])/X_std[Scale_cols]
    test_X = torch.Tensor(test_X.fillna(0).values.reshape(-1,22,train_X.shape[-1]))
    y_pred_p = np.zeros(199)
    for i,model in enumerate(model_list):
        model.eval()
        pred = model(test_X)[1][-1].detach().numpy()
        y_pred_p += pred / len(model_list)
    y_pred_p = np.clip(y_pred_p,0,1)
    y_pred_p[:int(99-yardline)] = 0
    y_pred_p[int(199-yardline):] = 1
    env.predict(pd.DataFrame(data=[y_pred_p],columns=sample_prediction_df.columns))
env.write_submission_file()


Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
